In [1]:
# imports
import requests
import json
import pandas as pd
import os

read data from bike stations CSV

In [2]:
bike_stations_df = pd.read_csv('bike_stations.csv', index_col = 0)

In [3]:
bike_stations_df

,Station Name,Available Bikes,Empty Slots,Latitude,Longitude
0,SE 47th at Hawthorne,1,2,45.511950,-122.614160
1,NE 50th at Fremont,0,6,45.548276,-122.611164
2,NE Cully at Skidmore,2,3,45.553917,-122.602071
3,SE Belmont at 45th,1,2,45.516585,-122.616038
4,SE 37th at Gladstone,1,2,45.493337,-122.623397
...,...,...,...,...,...
233,SE Alder at 6th,4,6,45.517899,-122.660052
234,SE 8th at Alder,8,0,45.518103,-122.657650
235,Cleveland High School,2,7,45.498947,-122.637669
236,NE 37th at NE Shaver,2,4,45.552358,-122.625588


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

foursquare_api_key = os.getenv('FOURSQUARE_API_KEY')

Get data from API calls using API key

In [5]:
def query_foursquare_for_places(lat, lng, radius=1000, limit = 50):
    url = f"https://api.foursquare.com/v3/places/search"
    FOURSQUARE_API_VERSION = '20230820'
    params = {
        'v': FOURSQUARE_API_VERSION,
        'll': f"{lat},{lng}",
        'radius': radius
    }
    
      
    # Create dictionary for headers
    headers = {"Accept": "application/json"}
    # Add key with our API KEY
    headers['Authorization'] = foursquare_api_key

    try:
        response = requests.get(url, headers=headers, params=params)
        #print(response)
        #print("Response content:", response.content)  # Print the raw response content
        response_data = response.json()
        #print(response.json())
        
        if 'results' in response_data:
            places = response_data['results']
            return places
        else:
            print("No Place of Interest found")
            return []         

    except requests.exceptions.RequestException as e:
        print("Error making API request:", e)
        return []

Get bike stations data from foursquare api call

In [6]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        query_foursquare_for_places(row['Latitude'], row['Longitude'])

Station Name: SE 47th at Hawthorne
Available Bikes: 1
Empty Slots: 2
Latitude: 45.51195046
Longitude: -122.6141596
Station Name: NE 50th at Fremont
Available Bikes: 0
Empty Slots: 6
Latitude: 45.54827571
Longitude: -122.6111636
Station Name: NE Cully at Skidmore
Available Bikes: 2
Empty Slots: 3
Latitude: 45.553917
Longitude: -122.6020709
Station Name: SE Belmont at 45th
Available Bikes: 1
Empty Slots: 2
Latitude: 45.51658489
Longitude: -122.6160378
Station Name: SE 37th at Gladstone
Available Bikes: 1
Empty Slots: 2
Latitude: 45.49333654
Longitude: -122.6233971
Station Name: SE Gideon at 12th Ave MAX station
Available Bikes: 11
Empty Slots: 0
Latitude: 45.50280906
Longitude: -122.6535398
Station Name: SW 10th at Harvey Milk
Available Bikes: 2
Empty Slots: 14
Latitude: 45.52175423
Longitude: -122.6810794
Station Name: SE Elliott at Division
Available Bikes: 11
Empty Slots: 6
Latitude: 45.50512504
Longitude: -122.6533681
Station Name: SE Clinton at 26th
Available Bikes: 1
Empty Slots: 5

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
def foursqare_places_of_interest(lat, lng, limit = 50):
    poi = query_foursquare_for_places(lat, lng, radius=1000)
    
    if poi:
        print("Nearby Places of Interest:")
        for places in poi:
            if 'coffee' in places['categories'][0]['name'].lower() or 'restaurant' in places['categories'][0]['name'].lower() or 'bar' in places['categories'][0]['name'].lower() or 'hotel' in places['categories'][0]['name'].lower():
                print(f"Business Category: {places['categories'][0]['name']}")
                if places['chains']:
                    print(f"Business Chain: {places['chains']}")
                print(f"Business Name: {places['name']}")
                print(f"Business Address: {places['location']['formatted_address']}")
                print("-" * 30)
    else:
        print("No Business found")

From responses get details for points of interest

In [8]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        
        foursqare_places_of_interest(row['Latitude'], row['Longitude'])

Station Name: SE 47th at Hawthorne
Available Bikes: 1
Empty Slots: 2
Latitude: 45.51195046
Longitude: -122.6141596
Nearby Places of Interest:
Business Category: American Restaurant
Business Name: Por Que No
Business Address: 4635 SE Hawthorne Blvd (at SE 47th Ave), Portland, OR 97215
------------------------------
Business Category: Chinese Restaurant
Business Name: The Whole Bowl
Business Address: 4409 SE Hawthorne Blvd (btwn SE 44th & 45th Ave.), Portland, OR 97215
------------------------------
Business Category: Fast Food Restaurant
Business Name: Kure Juice Bar
Business Address: 4409 SE Hawthorne Blvd (at SE 44th), Portland, OR 97215
------------------------------
Business Category: Coffee Shop
Business Name: Albina Press
Business Address: 5012 SE Hawthorne Blvd (btw 50th & 51st St), Portland, OR 97215
------------------------------
Business Category: Fast Food Restaurant
Business Name: Next Level Burger Hawthorne
Business Address: 4121 SE Hawthorne Blvd, Portland, OR 97214
------

Put your parsed results into a DataFrame

In [9]:
business = []

In [10]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        results = query_foursquare_for_places(row['Latitude'], row['Longitude'], limit = 50)
        
        for each in results:
            if each['categories']:
                Category = each['categories'][0]['name']
            if each['name']:
                Name = each['name']
            if each['location']:
                Location = each['location']['formatted_address']
            
                if 'coffee' in Category.lower() or 'restaurant' in Category.lower() or 'bar' in Category.lower() or 'hotel' in Category.lower():
                    business.append({'Station Name': row['Station Name'], 'Business Category': Category, 'Business Name': Name, 'Address': Location})

In [11]:
foursquare_business_df = pd.DataFrame(business)
foursquare_business_df

,Station Name,Business Category,Business Name,Address
0,SE 47th at Hawthorne,American Restaurant,Por Que No,"4635 SE Hawthorne Blvd (at SE 47th Ave), Portl..."
1,SE 47th at Hawthorne,Chinese Restaurant,The Whole Bowl,4409 SE Hawthorne Blvd (btwn SE 44th & 45th Av...
2,SE 47th at Hawthorne,Fast Food Restaurant,Kure Juice Bar,"4409 SE Hawthorne Blvd (at SE 44th), Portland,..."
3,SE 47th at Hawthorne,Coffee Shop,Albina Press,"5012 SE Hawthorne Blvd (btw 50th & 51st St), P..."
4,SE 47th at Hawthorne,Fast Food Restaurant,Next Level Burger Hawthorne,"4121 SE Hawthorne Blvd, Portland, OR 97214"
...,...,...,...,...
1010,NE 29th Ave (U of O),Cocktail Bar,Tough Luck,"1771 NE Dekum St (at NE 18th Ave), Portland, O..."
1011,NE 29th Ave (U of O),Fast Food Restaurant,Tamale Boy,"1764 NE Dekum St, Portland, OR 97211"
1012,NE 29th Ave (U of O),Wine Bar,Dame,"2930 NE Killingsworth St (NE 30th Ave), Portla..."
1013,NE 29th Ave (U of O),Cocktail Bar,Expatriate,"5424 NE 30th Ave (at NE Killingsworth St), Por..."


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [64]:
def query_yelp_for_places(lat, lng, radius = 1000, limit = 10):
    url = "https://api.yelp.com/v3/businesses/search"
    yelp_api_key = '5uKZ25JgcLYnowb3klyf2oWGdaXyutczk9t7FxY0Qnx4Pfqy4t5aR4ewP4TDvjxuUWusDK79tzj5JazZrGeJEb2X46U-Y8SQS_Av4hPNw6_UkDV_46trsDP-HSN5ZXYx'
    params = {
        "latitude": lat,
        "longitude": lng,
        "radius": radius
    }
    
    headers = {
        "Authorization": f"Bearer {yelp_api_key}"
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        #print(response)
        #print(response.content)
        response_data= response.json()
        
        if 'businesses' in response_data:
            places = response_data['businesses']
            return places
        else:
            print("No Place of Interest found")
            return []
    except requests.exceptions.RequestException as e:
        print("Error making API request:", e)
        return []

Get bike stations data from yelp api call

In [61]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        query_yelp_for_places(row['Latitude'], row['Longitude'])

Station Name: SE 47th at Hawthorne
Available Bikes: 1
Empty Slots: 2
Latitude: 45.51195046
Longitude: -122.6141596
Station Name: NE 50th at Fremont
Available Bikes: 0
Empty Slots: 6
Latitude: 45.54827571
Longitude: -122.6111636
Station Name: NE Cully at Skidmore
Available Bikes: 2
Empty Slots: 3
Latitude: 45.553917
Longitude: -122.6020709
Station Name: SE Belmont at 45th
Available Bikes: 1
Empty Slots: 2
Latitude: 45.51658489
Longitude: -122.6160378
Station Name: SE 37th at Gladstone
Available Bikes: 1
Empty Slots: 2
Latitude: 45.49333654
Longitude: -122.6233971
Station Name: SE Gideon at 12th Ave MAX station
Available Bikes: 11
Empty Slots: 0
Latitude: 45.50280906
Longitude: -122.6535398
Station Name: SW 10th at Harvey Milk
Available Bikes: 2
Empty Slots: 14
Latitude: 45.52175423
Longitude: -122.6810794
Station Name: SE Elliott at Division
Available Bikes: 11
Empty Slots: 6
Latitude: 45.50512504
Longitude: -122.6533681
Station Name: SE Clinton at 26th
Available Bikes: 1
Empty Slots: 5

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [65]:
def yelp_places_of_interest(lat, lng, limit = 50):
    poi = query_yelp_for_places(lat, lng, radius = 1000, limit = 50)
    
    if poi:
        print("Nearby Places of Interest:")
        for places in poi:
            print(f"Business Category: {places['categories'][0]['title']}")
            print(f"Business Name: {places['name']}")
            print(f"No of Reviews: {places['review_count']}")
            print(f"Rating: {places['rating']}")
            print(f"Business Address: {places['location']['display_address'][0]}")
            print(f"Business Contact: {places['display_phone']}")
            print("-" * 30)
    else:
        print("No Business found")

From responses get details for points of interest

In [66]:

if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        yelp_places_of_interest(row['Latitude'], row['Longitude'])

Station Name: SE 47th at Hawthorne
Available Bikes: 1
Empty Slots: 2
Latitude: 45.51195046
Longitude: -122.6141596
Nearby Places of Interest:
Business Category: Mexican
Business Name: Por Qué No? Taqueria
No of Reviews: 1365
Rating: 4.0
Business Address: 4635 SE Hawthorne Blvd
Business Contact: (503) 954-3138
------------------------------
Business Category: Pizza
Business Name: Apizza Scholls
No of Reviews: 1753
Rating: 4.0
Business Address: 4741 SE Hawthorne Blvd
Business Contact: (503) 233-1286
------------------------------
Business Category: Lounges
Business Name: Sapphire Hotel
No of Reviews: 492
Rating: 4.5
Business Address: 5008 SE Hawthorne Blvd
Business Contact: (503) 232-6333
------------------------------
Business Category: Noodles
Business Name: Baka Umai
No of Reviews: 178
Rating: 4.5
Business Address: 4703 SE Hawthorne Blvd
Business Contact: (971) 255-0116
------------------------------
Business Category: Thai
Business Name: Khao Thai
No of Reviews: 73
Rating: 4.5
Busine

Put your parsed results into a DataFrame

In [67]:
restaurants = []

In [68]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        results = query_yelp_for_places(row['Latitude'], row['Longitude'], radius = 1000, limit = 50)
        
        for each in results:
            if each['categories']:
                Category = each['categories'][0]['title']
            if each['name']:
                Name = each['name']
            if each['review_count']:
                Count_of_Reviews = each['review_count']
            if each['rating']:
                Rating = each['rating']
            if each['location']['display_address']:
                Address = each['location']['display_address'][0]
            if each['display_phone']:
                Contact = each['display_phone']
                
            restaurants.append({'Station Name': row['Station Name'], 'Station Latitude': row['Latitude'], 'Station Longitude': row['Longitude'], 'Business Category': Category, 'Business Name': Name, 'Rating': Rating, 'Number of Reviews': Count_of_Reviews, 'Address': Address, 'Contact': Contact})

In [69]:
yelp_business_df = pd.DataFrame(restaurants)

In [70]:
yelp_business_df

,Station Name,Station Latitude,Station Longitude,Business Category,Business Name,Rating,Number of Reviews,Address,Contact
0,SE 47th at Hawthorne,45.511950,-122.614160,Mexican,Por Qué No? Taqueria,4.0,1365,4635 SE Hawthorne Blvd,(503) 954-3138
1,SE 47th at Hawthorne,45.511950,-122.614160,Pizza,Apizza Scholls,4.0,1753,4741 SE Hawthorne Blvd,(503) 233-1286
2,SE 47th at Hawthorne,45.511950,-122.614160,Lounges,Sapphire Hotel,4.5,492,5008 SE Hawthorne Blvd,(503) 232-6333
3,SE 47th at Hawthorne,45.511950,-122.614160,Noodles,Baka Umai,4.5,178,4703 SE Hawthorne Blvd,(971) 255-0116
4,SE 47th at Hawthorne,45.511950,-122.614160,Thai,Khao Thai,4.5,73,4604 SE Hawthorne Blvd,(971) 346-2422
...,...,...,...,...,...,...,...,...,...
4749,NE 29th Ave (U of O),45.570366,-122.636043,Bakeries,Flour Market,5.0,28,5507 NE 30th Ave,(503) 209-5363
4750,NE 29th Ave (U of O),45.570366,-122.636043,Food Trucks,Honey Butter Country Fare,5.0,26,6719 NE 18th Ave,(503) 209-5363
4751,NE 29th Ave (U of O),45.570366,-122.636043,Coffee & Tea,Kiss Coffee,5.0,37,3016 NE Ainsworth St,(503) 209-5363
4752,NE 29th Ave (U of O),45.570366,-122.636043,Burgers,Bless Your Heart Burgers,4.0,132,5410 NE 33rd Ave,(503) 719-6447


In [71]:
yelp_business_df.to_csv('yelp_business.csv')

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp API provided me with a lot more data than Foursquare API. I was only able to extract three data points from Foursquare: The business name, category and location. Other data provided does not provide any value to the current exercise. On the other hand, Yelp offers additional information such as the number of reviews, the rating, type of transactions offered (pick up or delivery), the phone number to contact the business.

Get the top 10 restaurants according to their rating

In [72]:
Top_10_Restaurants = yelp_business_df.sort_values(by='Rating', ascending = False).head(10)
print(Top_10_Restaurants)

                     Station Name  Station Latitude  Station Longitude  \
2895       NE Dekum at Oneonta St         45.571781        -122.656484   
4163  NE Killingsworth at NE 52nd         45.562828        -122.609346   
4131        NE Halsey St at 106th         45.533440        -122.553860   
4135        NE Halsey St at 106th         45.533440        -122.553860   
2072     NE 42nd at Alberta Court         45.558361        -122.620361   
4149   N Rosa Parks Way at Albina         45.569917        -122.674420   
4152   N Rosa Parks Way at Albina         45.569917        -122.674420   
2069     NE 42nd at Alberta Court         45.558361        -122.620361   
2068     NE 42nd at Alberta Court         45.558361        -122.620361   
4155   N Rosa Parks Way at Albina         45.569917        -122.674420   

           Business Category                              Business Name  \
2895             Food Trucks                              Twirling Bird   
4163                American       